# this script builds a loop over all documents to extract the beccessary information

In [10]:
#load all neccessary packages
import pandas
import os
import requests
import re
import pickle
from bs4 import BeautifulSoup

In [12]:
#define the directory of the datafiles I want to loop over
user = '/Users/natalies_macbook/Documents/' #change this to your folder where GitHub sits
directory = user + 'GitHub/INSA/EU/relevant'

# basic pattern overview

#define all patterns
#get the document base Information       
url= r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"' #finds the url through the href tag
celex= r'uri=CELEX%([^"]+)' # get the celex id

def extract_publication_details(soup):
    publication_date = soup.find('p', class_='oj-hd-date').get_text(strip=True)
    title = soup.find('p', class_='oj-doc-ti').get_text(strip=True)
    return publication_date, title

#get the document information
decision_match = r'(COUNCIL.*?)</p'
legal_decision = r'<p[^>]*>(.*?(amending|implementing).*?)</p>'
legal_base = r'<p[^>]*>(.*?regard to.*?)</p>'
case_number = r'Case\sT-\d+/\d+'
law_number = r'<p class="oj-hd-oj">C (\d+/\d+)</p>'
legal_base_alt = r'\((CFSP|EU)\)\s*(\d+/\d+)'

#check for list updates, removals or deletions
action_intended = r'<p class="(oj-normal|oj-bold|normal)">\s*(.*?\s*(?:removed|updated|deleted|replaced)\s*.*?)\s*</p>'

#for annex with numbers
listing_position = r'class="oj-normal">(\d+.*?)'
name_2 = r'<p class="oj-normal">\s*<span class="oj-bold">(.*?)</span>'
un_sanction_date = r'class="oj-normal">Date of UN designation(.*?)</p>'
identifying_information = r'>(.*?Date of Birth:.*?)</p>'

UN_pattern = r'UN'

#court decision matching
name_pattern_court = r'Orders (.+?) to pay'
#for dismissal entity name

delist_pattern = r'Annuls'
delisting_case_pattern = r'Annuls (.+?);2.Orders'
name_pattern2 = r'in so far as they concern (.+?);2.Orders'
name_pattern1 = r'Orders (.+?) to pay'

# court decision matching

In [16]:
#court decision matching
name_pattern_court = r'Orders (.+?) to pay'
#for dismissal entity name
# Create patterns
delist_pattern = r'Annuls'
delisting_case_pattern = r'Annuls (.+?);2.Orders'
name_pattern2 = r'in so far as they concern (.+?);2.Orders'
name_pattern1 = r'Orders (.+?) to pay'

# Create a list to store all court decisions
court_decisions = []

# Iterate over the content
for i in content:
    # Get the content of the court case
    name = re.findall(r'<p class="oj-normal">(.*?)</p>', i)
    text = ''.join(name)
    
    # Initialize variables
    Delisting = None
    delisting_case = None
    entity_name = None
    
    # Check for delisting
    if re.search(delist_pattern, text):
        Delisting = 1
    else:
        Delisting = 0
    if Delisting == 1:
        match = re.search(name_pattern2, text)
        if match:
            entity_name = match.group(1)
    if Delisting == 0:
        match = re.search(name_pattern1, text)
        if match:
            entity_name = match.group(1)
    
    # Extract delisting case
    match = re.search(delisting_case_pattern, text)
    if match:
        delisting_case = match.group(1)

    # Create dictionary for current court decision
    court_decision = {
        'entity_name': entity_name,
        'Delisting': Delisting,
        'Delisting_case': delisting_case,
        'url': re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data),
        'celex' : re.findall(r'uri=CELEX%([^"]+)', data),
        'publication_date': re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data),
        'date': re.findall(r'<p class="oj-doc-ti">of (\d+ \w+ \d+)</p>', data),
        'title': re.findall(r'<p class="oj-doc-ti" id="[^"]*">(.*?)</p>', data),
        'law_number': re.findall(r'<p class="oj-hd-oj">C (\d+/\d+)</p>', data),
        'case_number': re.findall(r'Case\sT-\d+/\d+', data),
        'legal_base': re.findall(r'\((CFSP|EU)\)\s*(\d+/\d+)', data)
    }
    
    # Append the current decision dictionary to the list
    court_decisions.append(court_decision)

# Print the list of dictionaries
print(court_decisions, encoding = 'UTF-8') #encoding ändern oder string subsitute (string punkt replace xa0 mit .)
#now extract th information
# Initialize variables
Delisting = None
delisting_case = None
entity_name = None
#create patterns
delist_pattern = r'The Court: 1. Annuls'
delisting_case_pattern = r'Annuls (.+?);2.Orders'
name_pattern = r'in so far as they concern (.+?);2.Orders'
#create empty dic
results = {}

for i in content:
    #get the content of the court case
    name = re.findall(r'<p class="oj-normal">(.*?)</p>', i)
    text = ''.join(name)
    if re.search(delist_pattern, text):
        Delisting = 0
        print('Dismissal')
    else:
        Delisting = 1  # Assuming 1 for non-dismissal cases
    match = re.search(delisting_case_pattern, text)
    if match:
        delisting_case = match.group(1)
    # Extract entity name
    match = re.search(name_pattern, text)
    if match:
        entity_name = match.group(1)
#now get it into a dictionary format
court_decision = {}
court_decision['entity_name'] = entity_name
court_decision['Delisting'] = Delisting
court_decision['Delisting_case']= delisting_case
court_decision['url'] = re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data)
court_decision['publication_date'] =re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data)
court_decision['date'] =  re.findall(r'<p class="oj-doc-ti">of (\d+ \w+ \d+)</p>', data)
court_decision['title']= re.findall(r'<p class="oj-doc-ti" id="[^"]*">(.*?)</p>', data)
court_decision['law_number'] = re.findall(r'<p class="oj-hd-oj">C (\d+/\d+)',data)
court_decision['case_number'] = re.findall(r'Case\sT-\d+/\d+', data)
court_decision['legal_base'] = re.findall(r'\((CFSP|EU)\)\s*(\d+/\d+)', data)


NameError: name 'content' is not defined

In [ ]:
def extract_court_decision(data):
    # Define the patterns
    delist_pattern = r'Annuls'
    delisting_case_pattern = r'Annuls (.+?);2.Orders'
    name_pattern2 = r'in so far as they concern (.+?);2.Orders'
    name_pattern1 = r'Orders (.+?) to pay'
    
    # Initialize variables
    Delisting = 0  # Default to 0 (not a delisting)
    delisting_case = None
    entity_name = None
    
    # Get the content of the court case
    name = re.findall(r'<p class="oj-normal">(.*?)</p>', data)
    text = ''.join(name)
    
    # Check for delisting
    if re.search(delist_pattern, text):
        Delisting = 1  # Set to 1 if it matches the delisting pattern
        match = re.search(name_pattern2, text)
        if match:
            entity_name = match.group(1)
    else:
        match = re.search(name_pattern1, text)
        if match:
            entity_name = match.group(1)
    
    # Extract delisting case
    match = re.search(delisting_case_pattern, text)
    if match:
        delisting_case = match.group(1)
    
    # Create a dictionary for the current court decision
    return {
        'entity_name': entity_name,
        'Delisting': Delisting,  # Binary indicator (1 for delisting, 0 otherwise)
        'Delisting_case': delisting_case,
        'url': re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data),
        'celex': re.findall(r'uri=CELEX%([^"]+)', data),
        'publication_date': re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data),
        'date': re.findall(r'<p class="oj-doc-ti">of (\d+ \w+ \d+)</p>', data),
        'title': re.findall(r'<p class="oj-doc-ti" id="[^"]*">(.*?)</p>', data),
        'law_number': re.findall(r'<p class="oj-hd-oj">C (\d+/\d+)</p>', data),
        'case_number': re.findall(r'Case\sT-\d+/\d+', data),
        'legal_base': re.findall(r'\((CFSP|EU)\)\s*(\d+/\d+)', data)
    }

# patterns for notices

In [ ]:
def notice_pattern_extraction(soup):
    # Document base information
    url_pattern = r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"'
    celex_pattern = r'uri=CELEX%([^"]+)'
    
    # Extract publication date and title
    publication_date = soup.find('p', class_='oj-hd-date')
    publication_date = publication_date.get_text(strip=True) if publication_date else None
    
    title = soup.find('p', class_='oj-doc-ti')
    title = title.get_text(strip=True) if title else None
    
    # Extract URL and CELEX
    url_match = re.search(url_pattern, str(soup))
    celex_match = re.search(celex_pattern, str(soup))
    url = url_match.group(1) if url_match else None
    celex = celex_match.group(1) if celex_match else None
    
    # Document information patterns
    decision_match_pattern = r'(Decision[^,]*|Regulation[^,]*|Annex[^,]*)'
    legal_action_pattern = r'<p[^>]*>(.*?(implementing|amended|implemented).*?)</p>'
    legal_base_pattern = r'<p[^>]*>(.*?regard to|amending.*?)</p>'
    case_number_pattern = r'Case\sT-\d+/\d+'
    law_number_pattern = r'<p class="oj-hd-oj">C (\d+/\d+)</p>'
    legal_base_alt_pattern = r'\((CFSP|EU)\)\s*(\d+/\d+)'
    
    decision_match = re.search(decision_match_pattern, str(soup))
    legal_decision = re.search(legal_decision_pattern, str(soup))
    legal_base = re.search(legal_base_pattern, str(soup))
    case_number = re.search(case_number_pattern, str(soup))
    law_number = re.search(law_number_pattern, str(soup))
    legal_base_alt = re.search(legal_base_alt_pattern, str(soup))
    
    # List updates, removals, deletions
    action_intended_pattern = r'<p class="(oj-normal|oj-bold|normal|bold)">\s*(.*?\s*(?:removed|updated|deleted|replaced|amended)\s*.*?)\s*</p>'
    action_intended = re.search(action_intended_pattern, str(soup))
    notice_target = re.findall(r'The following information is brought to the attention of\s*(.*?),', str(soup))
    
    return {'publication_date': publication_date,
        'title': title,
        'url': url,
        'celex': celex,
        'decision_match': decision_match.group(1) if decision_match else None,
        'legal_decision': legal_decision.group(1) if legal_decision else None,
        'legal_base': legal_base.group(1) if legal_base else None,
        'case_number': case_number.group(0) if case_number else None,
        'law_number': law_number.group(1) if law_number else None,
        'legal_base_alt': legal_base_alt.group(0) if legal_base_alt else None,
        'action_intended': action_intended.group(2) if action_intended else None,
        'notice_target':notice_target.group(1) if notice_target else None}


# main pattern all

this one works so far but i have to include the number of the notice if it is a notice:
    oj-no-doc-c
    

as well as fix the identifying info list, i cant pluck from the position of the returned matches

this is what it looks like:

                     </td>
                     <td valign="top" class="oj-table">
                        <p class="oj-tbl-txt">2 March 2021</p>
                     </td>
                  </tr>
                  <tr class="oj-table">
                     <td valign="top" class="oj-table">
                        <p class="oj-tbl-txt">3.</p>
                     </td>
                     <td valign="top" class="oj-table">
                        <p class="oj-tbl-txt">Igor Viktorovich KRASNOV</p>
                     </td>
                     <td valign="top" class="oj-table">
                        <p class="oj-tbl-txt">Игорь Викторович КРАСНОВ</p>
                     </td>
                     <td valign="top" class="oj-table">
                        <p class="oj-tbl-txt">Position(s): Prosecutor General of the Russian Federation</p>
                        <p class="oj-tbl-txt">DOB: 24.12.1975</p>
                        <p class="oj-tbl-txt">POB: Arkhangelsk, Russian SFSR (now Russian Federation)</p>
                        <p class="oj-tbl-txt">Nationality: Russian</p>
                        <p class="oj-tbl-txt">Gender: male</p>
                     </td>
                     <td valign="top" class="oj-table">
                        <p class="oj-tbl-txt">Igor Krasnov has been Prosecutor General of the Russian Federation since 22 January 2020, and is the former Deputy Chairman of the Investigative Committee of the Russian Federation. In his position as Prosecutor General, he supervises the Prosecutor’s Offices in the Russian Federation, the Special Prosecutor’s Offices and the Military Prosecutor’s Office. In his capacity as Prosecutor General, he is responsible for serious human rights violations, including the arbitrary detentions of protesters, and for widespread and systematic repression of freedom of peaceful assembly and of association, and freedom of opinion and expression.</p>
                        <p class="oj-tbl-txt">Ahead of the 23 January 2021 protests, the Prosecutor General’s Office warned that participants would be held responsible. Moreover, the Prosecutor General’s Office demanded that the Federal Service for Supervision in the Sphere of Communications, Information Technology and Mass Communications (Roskomnadzor) restrict access to opposition websites and social network accounts that contained information on planned gatherings of Alexei Navalny’s supporters. On 29 January 2021, the Prosecutor General’s Office once again demanded that Roskomnadzor restrict access to opposition websites and social network accounts, this time ahead of the pro-Navalny protests on 30 and 31 January 2021. Warnings were sent to internet companies (Facebook, TikTok, Twitter, Google, Mail.ru Group). The Prosecutor General’s Office also announced that those taking part in the demonstrations would be prosecuted.</p>
                        <p class="oj-tbl-txt">The Prosecutor General’s Office supported the request by the Russian Federal Penitentiary Service (FSIN) to convert the suspended sentence imposed on Alexei Navalny in a case of alleged fraud to a prison sentence. Despite the fact that his conviction in that case had been found arbitrary and unfair by the European Court of Human Rights in 2018, Alexei Navalny was arrested upon his arrival at Moscow airport on 17 January 2021.</p>
                     </td>
                     <td valign="top" class="oj-table">
                        <p class="oj-tbl-txt">2 March 2021</p>
                     </td>
                  </tr>
                  <tr class="oj-table">
                     <td valign="top" class="oj-table">
                        <p class="oj-tbl-txt">4.</p>
                     </td>

In [ ]:
#change the identifying information regex

name = cells[1].get_text(strip=True) if cells[1] else None #can keep this
alias = cells[2].get_text(strip=True) if cells[2] else None #can keep this
                identifying_info = cells[3].get_text(strip=False) if cells[3] else None  # Extracting the identifying information but keeping the newline 
                reasons = ' '.join(p.get_text(strip=True) for p in cells[3].find_all('p')) if cells[3] else None
                date_of_listing = cells[4].get_text(strip=True) if cells[4] else None
                
identifying_information_pattern = r'>(.*?Date of Birth:.*?)</p>'


# write patterns to extract the inform ation from the identifying information column
position_pattern ='<p class="oj-tbl-txt">(Position(s):(.*?))</p>'
'Born on'#possible match
dob_pattern='<p class="oj-tbl-txt">DOB|Date of Birth: {1,2}/.{1,2}./{1,4}</p>'
pob_pattern='<p class="oj-tbl-txt">POB|Place of Birth:(.*?)</p>'
nationality_pattern='<p class="oj-tbl-txt">Nationality:(.*?)</p>'
gender_pattern='<p class="oj-tbl-txt">Gender:(.*?)</p>'
passport_pattern='<p class="oj-tbl-txt">Passport Number:(.*?)</p>'
'Social media:
Email: 
Telephone
Address:
Code:
    #add a match for everything that is not matched in the above


In [ ]:
#court decision pattern needs to be updated

In [44]:

#paterns for the general listing documents
def extract_document_info(soup):
    # Document base information
    url_pattern = r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"'
    celex_pattern = r'uri=CELEX%([^"]+)'
    
    # Extract publication date and title
    publication_date = soup.find('p', class_='oj-hd-date')
    publication_date = publication_date.get_text(strip=True) if publication_date else None
    
    title = soup.find('p', class_='oj-doc-ti')
    title = title.get_text(strip=True) if title else None
    
    # Extract URL and CELEX
    url_match = re.search(url_pattern, str(soup))
    celex_match = re.search(celex_pattern, str(soup))
    url = url_match.group(1) if url_match else None
    celex = celex_match.group(1) if celex_match else None
    
    # Document information patterns
    decision_match_pattern = r'(Decision[^,]*|Regulation[^,]*|Annex[^,]*)'
    legal_action_pattern = r'<p[^>]*>(.*?(implementing|amended|implemented).*?)</p>'
    legal_base_pattern = r'<p[^>]*>(.*?regard to|amending.*?)</p>'
    case_number_pattern = r'Case\sT-\d+/\d+'
    law_number_pattern = r'<p class="oj-hd-oj">C (\d+/\d+)</p>'
    legal_base_alt_pattern = r'\((CFSP|EU)\)\s*(\d+/\d+)'
    
    decision_match = re.search(decision_match_pattern, str(soup))
    legal_decision = re.search(legal_decision_pattern, str(soup))
    legal_base = re.search(legal_base_pattern, str(soup))
    case_number = re.search(case_number_pattern, str(soup))
    law_number = re.search(law_number_pattern, str(soup))
    legal_base_alt = re.search(legal_base_alt_pattern, str(soup))
    
    # List updates, removals, deletions
    action_intended_pattern = r'<p class="(oj-normal|oj-bold|normal|bold)">\s*(.*?\s*(?:removed|updated|deleted|replaced|amended)\s*.*?)\s*</p>'
    action_intended = re.search(action_intended_pattern, str(soup))
    
    # Annex with numbers
    listing_position_pattern = r'class="oj-normal|normal">(\d+.*?)'
    name_2_pattern = r'<p class="oj-normal|normal">\s*<span class="oj-bold|bold">(.*?)</span>'
    un_sanction_date_pattern = r'class="oj-normal|normal">Date of UN designation(.*?)</p>'
    identifying_information_pattern = r'>(.*?Date of Birth:.*?)</p>'
    
    listing_position = re.search(listing_position_pattern, str(soup))
    name_2 = re.search(name_2_pattern, str(soup))
    un_sanction_date = re.search(un_sanction_date_pattern, str(soup))
    identifying_information = re.search(identifying_information_pattern, str(soup)
    
    # Court decision matching
    #name_pattern_court = r'Orders (.+?) to pay'
    #delist_pattern = r'Annuls'
    #delisting_case_pattern = r'Annuls (.+?);2.Orders'
    #name_pattern2 = r'in so far as they concern (.+?);2.Orders'
    #name_pattern1 = r'Orders (.+?) to pay'

    #name_court = re.search(name_pattern_court, str(soup))
    #delist_match = re.search(delist_pattern, str(soup))
    #delisting_case = re.search(delisting_case_pattern, str(soup))
    #name_case2 = re.search(name_pattern2, str(soup))
    #name_case1 = re.search(name_pattern1, str(soup))
    
    # Return all extracted information
    return {
        'publication_date': publication_date,
        'title': title,
        'url': url,
        'celex': celex,
        'decision_match': decision_match.group(1) if decision_match else None,
        'legal_decision': legal_decision.group(1) if legal_decision else None,
        'legal_base': legal_base.group(1) if legal_base else None,
        'case_number': case_number.group(0) if case_number else None,
        'law_number': law_number.group(1) if law_number else None,
        'legal_base_alt': legal_base_alt.group(0) if legal_base_alt else None,
        'action_intended': action_intended.group(2) if action_intended else None,
        'listing_position': listing_position.group(1) if listing_position else None,
        'name_2': name_2.group(1) if name_2 else None,
        'un_sanction_date': un_sanction_date.group(1) if un_sanction_date else None,
        'identifying_information': identifying_information.group(1) if identifying_information else None,
        #'name_court': name_court.group(1) if name_court else None,
        #'delist_match': delist_match.group(0) if delist_match else None,
        #'delisting_case': delisting_case.group(1) if delisting_case else None,
        #'name_case2': name_case2.group(1) if name_case2 else None,
        #'name_case1': name_case1.group(1) if name_case1 else None,
    }
print('successfully memorized the listing patterns)

Processed batch 1 with 500 files.
Processed batch 2 with 500 files.
Processed batch 3 with 500 files.
Processed batch 4 with 500 files.
Processed batch 5 with 498 files.


In [ ]:
#function to process the file in batches
def process_files_in_batches(directory, batch_size=500):
    filenames = [f for f in os.listdir(directory) if f.endswith('.txt')]
    total_files = len(filenames)
    
    sanction_list = []
    
    for i in range(0, total_files, batch_size):
        batch_filenames = filenames[i:i+batch_size]
        
        for filename in batch_filenames:
            file_path = os.path.join(directory, filename)
            try:
                with open(file_path, encoding='utf-8') as f:
                    data = f.read()
            except UnicodeDecodeError:
                with open(file_path, encoding='latin-1') as f:
                    data = f.read()
                    
            soup = BeautifulSoup(data, 'html.parser')                         
            document_info = extract_document_info(soup)  # Extract document-level information
            
            if re.findall(court_decision_pattern, str(soup)): #matches all documents that are court decisions
                court_info = extract_court_decision(soup)
                court_decision_list =[]
                court_decision_list.append(court_info)
            else:
                continue
                
            if re.findall(notice_pattern, str(soup)): #matches all documents that are notices
                                        notice_info = notice_pattern_extraction(soup)
                                        notice_list = []
                                        notice = {**notice_info}
                                        notice_list.append(notice)
            else:
                                        continue

                                        
            rows = soup.find_all('tr', class_='oj-table')  # Finds all elements tr = tablerow of the class oj-table
            for row in rows[1:]:  # Skip the header row
                cells = row.find_all('td')  # Finds all td table data in one tablerow
                if len(cells) < 6:
                    continue
                name = cells[1].get_text(strip=True) if cells[1] else None
                alias = cells[2].get_text(strip=True) if cells[2] else None
                identifying_info = cells[3].get_text(strip=False) if cells[3] else None  # Extracting the identifying information but keeping the newline 
                reasons = ' '.join(p.get_text(strip=True) for p in cells[3].find_all('p')) if cells[3] else None
                date_of_listing = cells[4].get_text(strip=True) if cells[4] else None
                
                # Combine document_info with row-specific information
                sanction_dict = {
                    **document_info,
                    'Name': name,
                    'Alias': alias,
                    'Identifying Information': identifying_info,
                    'Reason': reasons,
                    'Date of Listing': date_of_listing
                }
                sanction_list.append(sanction_dict)
        
        # Optionally, save or process the current batch
        print(f"Processed batch {i//batch_size + 1} with {len(batch_filenames)} files.")

        # Clear sanction_list if saving batches to disk or if memory usage is a concern
        # sanction_list.clear()

    return sanction_list
    return notice_list

In [ ]:


# Process files in batches
sanctions = process_files_in_batches(directory, batch_size=500)

# pickling the results

In [14]:

with open("batched_sanction_EU", "wb") as fp:   #Pickling
    pickle.dump(sanction_list, fp)

with open("batched_notice_EU", "wb") as fp:   #Pickling
    pickle.dump(notice_list, fp)

NameError: name 'sanctions' is not defined

In [20]:

#with open("batched_sanction_EU", "rb") as fp:   #Pickling
#    pickle.load(fp)

EOFError: Ran out of input